# Exportacion ONNX para Transformers.js (Qwen2.5 fine-tuned)

Este cuaderno exporta el modelo merged a ONNX (formato oficial de Transformers.js) usando Optimum.
Se ejecuta en Google Colab.

In [ ]:
# Instalar dependencias
!pip -q install 'optimum[exporters]' onnx onnxruntime transformers


## Subir el modelo merged
Sube la carpeta `epicrisis-merged/` (zip) desde tu maquina.

In [ ]:
from google.colab import files
uploaded = files.upload()
# Espera un archivo epicrisis-merged.zip


In [ ]:
!unzip -q epicrisis-merged.zip
!ls -la


## Exportar a ONNX (opset 18)
Se usa `text-generation-with-past` para KV-cache.

In [ ]:
!optimum-cli export onnx \
  --model ./epicrisis-merged \
  --task text-generation-with-past \
  --dtype fp32 \
  --opset 18 \
  --no-dynamic-axes \
  --batch_size 1 \
  --sequence_length 2 \
  ./epicrisis-finetuned-onnx


## Reorganizar estructura para Transformers.js
Mueve ONNX a `onnx/` y renombra a `model_fp16.onnx` si aplica.

In [ ]:
import os, shutil, json
out_dir = './epicrisis-finetuned-onnx'
onnx_dir = os.path.join(out_dir, 'onnx')
os.makedirs(onnx_dir, exist_ok=True)
for f in os.listdir(out_dir):
    if f.endswith('.onnx') or f.endswith('.onnx_data'):
        src = os.path.join(out_dir, f)
        dst = os.path.join(onnx_dir, f)
        shutil.move(src, dst)

# Agregar transformers.js_config basico
cfg_path = os.path.join(out_dir, 'config.json')
if os.path.exists(cfg_path):
    with open(cfg_path, 'r') as f:
        cfg = json.load(f)
    cfg['transformers.js_config'] = {
        'dtype': 'fp32',
        'kv_cache_dtype': {
            'fp32': 'float32'
        }
    }
    with open(cfg_path, 'w') as f:
        json.dump(cfg, f, indent=2)

print('Listo:', out_dir)


## Empaquetar salida
Descarga la carpeta generada.

In [ ]:
!zip -r epicrisis-finetuned-onnx.zip epicrisis-finetuned-onnx
files.download('epicrisis-finetuned-onnx.zip')
